In [3]:
### 데이터 불러오기
import pandas as pd

X_train = pd.read_csv('./Practice/data/연습문제/FIFA_X_train.csv', encoding = 'cp949')
y_train = pd.read_csv('./Practice/data/연습문제/FIFA_y_train.csv', encoding = 'cp949')
X_test = pd.read_csv('./Practice/data/연습문제/FIFA_X_test.csv', encoding = 'cp949')


### 데이터 전처리
# 1. 불필요 변수 ID, Jersey_Number, Club 제거
X_train = X_train.drop(columns = 'ID')
y_train = y_train.drop(columns = 'ID')
ID = X_test.pop('ID')

X_train = X_train.drop(columns = 'Jersey_Number')
X_test = X_test.drop(columns = 'Jersey_Number')

X_train = X_train.drop(columns = 'Club')
X_test = X_test.drop(columns = 'Club')

# 2. Age 변수 그룹화하기
X_train['Age_group'] = X_train['Age'].str[0] + '0'
X_train['Age_group'] = X_train['Age_group']

X_test['Age_group'] = X_test['Age'].str[0] + '0'
X_test['Age_group'] = X_test['Age_group']

X_train = X_train.drop(columns = 'Age')
X_test = X_test.drop(columns = 'Age')

# print(X_train.isna().sum(), '\n\n\n')
# print(X_test.isna().sum())
    # 결측치 Position_Class 758, Height_cm 292, Weight_lb 72

# 3. 결측치 처리
X_train['Position_Class'].fillna('unknown', inplace = True)
X_test['Position_Class'].fillna('unknown', inplace = True)

cross_train = pd.crosstab(index = X_train['Position'], columns = X_train['Position_Class'])
cross_test = pd.crosstab(index = X_test['Position'], columns = X_test['Position_Class'])
# print(cross_train)
# print(cross_test)

PC_train = X_train['Position_Class'].copy()
PC_train[X_train['Position'] == 'CM'] = 'Midfielder'
PC_train[X_train['Position'] == 'GK'] = 'GoalKeeper'
PC_train[X_train['Position'] == 'RDM'] = 'Defender'
PC_train[X_train['Position'] == 'RWB'] = 'Defender'
X_train['Position_Class'] = PC_train

PC_test = X_test['Position_Class'].copy()
PC_test[X_test['Position'] == 'CM'] = 'Midfielder'
PC_test[X_test['Position'] == 'GK'] = 'GoalKeeper'
PC_test[X_test['Position'] == 'RDM'] = 'Defender'
PC_test[X_test['Position'] == 'RWB'] = 'Defender'

X_train = X_train.drop(columns = 'Position')
X_test = X_test.drop(columns = 'Position')

# print(X_train['Height'].head(10))

p_train = X_train['Height'].str.split("'", expand = True).astype('float64')
H_train = p_train[0]*30 + p_train[1]*2.5
X_train['Height_cm'] = H_train

p_test = X_test['Height'].str.split("'", expand = True).astype('float64')
H_test = p_test[0]*30 + p_test[1]*2.5
X_test['Height_cm'] = H_test

X_train = X_train.drop(columns = 'Height')
X_test = X_test.drop(columns = 'Height')

index_na = X_train[X_train['Weight_lb'].isna() == True].index

X_train = X_train.drop(index = index_na)
y_train = y_train.drop(index = index_na)
        # 결측치 처리 결과 ---> Position_Class 값 부여, Height_cm 값 부여, Weight_lb 결측 행 삭제


        # Nationality, Overall, Preferred_Foot, Work_Rate, Position_Class, Contract_Valid_Until, 
        # Height_cm, Weight_lb, Release_Clause, Wage, Age_group

# 4. Work_Rate --> Offense, Defense 분리
Work_train = X_train['Work_Rate'].copy().str.replace(' ','')  # 공백 제거
Work_test = X_test['Work_Rate'].copy().str.replace(' ','')  # 공백 제거

Work_train = Work_train.str.split('/', expand = True)
X_train['Offense'] = Work_train[0]
X_train['Defense'] = Work_train[1]

Work_test = Work_test.str.split('/', expand = True)
X_test['Offense'] = Work_test[0]
X_test['Defense'] = Work_test[1]

X_train = X_train.drop(columns = 'Work_Rate')
X_test = X_test.drop(columns = 'Work_Rate')
# print(X_train.info(), '\n\n\n')
# print(y_train.info(), '\n\n\n')
# print(X_test.info())
        # Nationality, Overall, Preferred_Foot, Work_Rate, Position_Class, Contract_Valid_Until, 
        # Height_cm, Weight_lb, Release_Clause, Wage, Age_group, Offense, Deffense
        
# print(X_train['Contract_Valid_Until'].sort_values().unique())
X_train['CVU_group'] = X_train['Contract_Valid_Until'].astype('object')
X_test['CVU_group'] = X_test['Contract_Valid_Until'].astype('object')

X_train = X_train.drop(columns = 'Contract_Valid_Until')
X_test = X_test.drop(columns = 'Contract_Valid_Until')

# 5. 상관분석
columns = ['Overall','Height_cm','Weight_lb','Release_Clause', 'Wage']
# print(X_train[columns].corr())
        # Release_Clause와 Wage의 상관관계가 크군. 하나 지워도 되겠어.

X_train = X_train.drop(columns = 'Release_Clause')
X_test = X_test.drop(columns = 'Release_Clause')


### 데이터 분할
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  test_size = 0.3,
                                                  random_state = 2023)
# print(X_train.shape)
# print(y_train.shape)
# print(X_val.shape)
# print(y_val.shape)

### 인코딩
from sklearn.preprocessing import OneHotEncoder

X_train_obj = X_train.select_dtypes('object').copy()
X_val_obj = X_val.select_dtypes('object').copy()
X_test_obj = X_test.select_dtypes('object').copy()

oh = OneHotEncoder(handle_unknown = 'ignore', sparse = False).fit(X_train_obj)

X_train_oh = oh.transform(X_train_obj)
X_val_oh = oh.transform(X_val_obj)
X_test_oh = oh.transform(X_test_obj)

### 스케일링
from sklearn.preprocessing import StandardScaler

columns = ['Overall','Height_cm','Weight_lb', 'Wage']

X_train_num = X_train[columns].copy()
X_val_num = X_val[columns].copy()
X_test_num = X_test[columns].copy()

scale = StandardScaler().fit(X_train_num)

X_train_scale = scale.transform(X_train_num)
X_val_scale = scale.transform(X_val_num)
X_test_scale = scale.transform(X_test_num)

### 데이터셋 준비
import numpy as np

X_train = np.concatenate([X_train_oh, X_train_scale], axis = 1)
X_val = np.concatenate([X_val_oh, X_val_scale], axis = 1)
X_test = np.concatenate([X_test_oh, X_test_scale], axis = 1)

y_train = y_train.values.ravel()
y_val = y_val.values.ravel()


### 모델 학습
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor

# 랜덤포레스트
rf = RandomForestRegressor(n_estimators = 500, 
                           max_depth = 3, 
                           min_samples_leaf = 10, 
                           max_features = 50, 
                           random_state = 2023)

model_rf = rf.fit(X_train, y_train)

# 배깅
dtr = DecisionTreeRegressor(max_depth = 3, 
                            min_samples_leaf = 10)
bag = BaggingRegressor(base_estimator = dtr, 
                       n_estimators = 500, 
                       random_state = 2023)

model_bag = bag.fit(X_train, y_train)

# 에이다부스트
dtr = DecisionTreeRegressor(max_depth = 3, 
                            min_samples_leaf = 10)
ada = AdaBoostRegressor(base_estimator = dtr, 
                        n_estimators = 500, 
                        learning_rate = 0.5, 
                        random_state = 2023)

model_ada = ada.fit(X_train, y_train)


### 성능 평가
from sklearn.metrics import mean_squared_error

pred_rf = model_rf.predict(X_val)
pred_bag = model_bag.predict(X_val)
pred_ada = model_ada.predict(X_val)

# rmse_rf = mean_squared_error(y_val, pred_rf, squared = False)
# print(rmse_rf)

# rmse_bag = mean_squared_error(y_val, pred_bag, squared = False)  # 당첨 !
# print(rmse_bag)

# rmse_ada = mean_squared_error(y_val, pred_ada, squared = False)
# print(rmse_ada)

        # 배깅의 RMSE가 가장 작으므로 배깅 모델 선정 !

### 결과 제출    
y_pred = model_bag.predict(X_test)

obj = {'ID':ID, 'Value':y_pred}

result = pd.DataFrame(obj)
result.to_csv("정식.csv", index = False)


### 모델 평가
actual = pd.read_csv('./Practice/data/연습문제/FIFA_y_test.csv', encoding = 'cp949')
actual = actual['Value'].ravel()

1801.9728870017418
7508306.855777261


In [18]:
### 데이터 불러오기
import pandas as pd

X_train = pd.read_csv('./Practice/data/연습문제/College_X_train.csv', encoding = 'cp949')
y_train = pd.read_csv('./Practice/data/연습문제/College_y_train.csv', encoding = 'cp949')
X_test = pd.read_csv('./Practice/data/연습문제/College_X_test.csv', encoding = 'cp949')

# print(X_train.head(), '\n\n\n')
# print(X_train.info(), '\n\n\n')
# print(X_train.describe(), '\n\n\n')
# print(X_train.isna().sum())  # 결측치 0

    # ID, Name, Apps, Accept, Enroll, Top10perc, Topperc, F.Undergrad, P.Undergrad, Outstate, 
    # Room.Board, Books, Personal, PhD, Terminal, S.F.Ratio, perc.alumni, Expend, Grad.Rate

### 데이터 전처리
# 불필요 변수 제거 ID, Name
X_train = X_train.drop(columns = ['ID','Name'])
y_train = y_train.drop(columns = 'ID')

ID = X_test.pop('ID')
X_test = X_test.drop(columns = 'Name')

# 수치형 변수 조정  Top10perc, Top25perc, PhD, Terminal, S.F.Ratio, perc.alumni, Grad.Rate
ratio_col = ['Top10perc', 'Top25perc', 'PhD', 'Terminal', 'S.F.Ratio', 'perc.alumni', 'Grad.Rate']
X_train[ratio_col] = X_train[ratio_col]/100
X_test[ratio_col] = X_test[ratio_col]/100

print(X_train.corr())  # Apps - Accept - Enroll - F.Undergrad - P.Undergrad // Top10perc - Top25perc // PhD - Terminal // Outstate - Expend
    # P.Undergrad, Top10perc, PhD, Outstate



                 Apps    Accept    Enroll  Top10perc  Top25perc  F.Undergrad  \
Apps         1.000000  0.944467  0.849168   0.349779   0.361195     0.821066   
Accept       0.944467  1.000000  0.916140   0.197776   0.252846     0.883243   
Enroll       0.849168  0.916140  1.000000   0.187864   0.231857     0.965347   
Top10perc    0.349779  0.197776  0.187864   1.000000   0.888509     0.151740   
Top25perc    0.361195  0.252846  0.231857   0.888509   1.000000     0.203923   
F.Undergrad  0.821066  0.883243  0.965347   0.151740   0.203923     1.000000   
P.Undergrad  0.436428  0.497542  0.558695  -0.119721  -0.074995     0.623587   
Outstate     0.053210 -0.028377 -0.150202   0.562420   0.497475    -0.202082   
Room.Board   0.165495  0.091953 -0.038736   0.392123   0.350296    -0.065608   
Books        0.130910  0.107125  0.089683   0.176155   0.165195     0.085500   
Personal     0.160231  0.176265  0.253702  -0.087108  -0.086286     0.291877   
PhD          0.386766  0.349032  0.32813